# 🚀 Complete LangGraph Tutorial: From Basics to Advanced Applications

## 📖 Comprehensive Guide for Building Stateful AI Agents

Welcome to the most comprehensive LangGraph tutorial! This notebook will take you from complete beginner to building sophisticated AI agents step by step.

### 🎯 What You'll Learn

By the end of this tutorial, you will:
- ✅ Understand LangGraph's core concepts (Nodes, Edges, State)
- ✅ Build your first simple chatbot
- ✅ Add memory and persistence to conversations
- ✅ Implement tool calling and external integrations
- ✅ Create human-in-the-loop workflows
- ✅ Build multi-agent systems
- ✅ Handle complex state management
- ✅ Deploy production-ready applications

### 📚 Based on Official LangGraph Documentation

This tutorial follows the [official LangGraph documentation](https://langchain-ai.github.io/langgraph/) and incorporates best practices from the LangGraph team.

---

## 📋 Table of Contents

1. **[Setup & Installation](#setup)**
2. **[Part 1: Understanding LangGraph Fundamentals](#part1)**
3. **[Part 2: Building Your First Simple Agent](#part2)**
4. **[Part 3: Adding Memory with Checkpointing](#part3)**
5. **[Part 4: Tool Integration & External APIs](#part4)**
6. **[Part 5: Human-in-the-Loop Workflows](#part5)**
7. **[Part 6: Advanced State Management](#part6)**
8. **[Part 7: Multi-Agent Systems](#part7)**
9. **[Part 8: Real-World Use Cases](#part8)**
10. **[Part 9: Production Deployment](#part9)**

Let's begin this exciting journey! 🌟


## 🔧 Setup & Installation {#setup}

Before we dive into LangGraph, let's set up our environment properly.

### Prerequisites

1. **Python 3.8+** installed on your system
2. **API Keys** for LLM providers (we'll use OpenAI in this tutorial)
3. **Basic Python knowledge** - understanding of functions, classes, and dictionaries

### What is LangGraph?

**LangGraph** is a library for building **stateful, multi-actor applications** with LLMs. It extends LangChain with the ability to coordinate multiple chains (or actors) across multiple steps of computation in a **cyclic** manner.

Key features:
- 🔄 **Cyclic workflows** (not just linear chains)
- 💾 **Persistent state** across interactions
- 🎯 **Conditional routing** between different paths
- 🔧 **Human-in-the-loop** capabilities
- 📊 **Built-in observability** with LangSmith

Think of it as a way to build AI agents that can:
- Remember previous conversations
- Make decisions about what to do next
- Use tools and external APIs
- Involve humans when needed
- Handle complex, multi-step workflows


In [ ]:
# Install required packages
!pip install -q langgraph langsmith langchain-openai python-dotenv

# Optional: for visualization
!pip install -q matplotlib graphviz


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

# Set up environment variables
def setup_environment():
    """Setup API keys for the tutorial"""
    
    # OpenAI API Key
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key = getpass.getpass("Enter your OpenAI API Key: ")
        os.environ["OPENAI_API_KEY"] = openai_key
    
    # Optional: LangSmith for observability (highly recommended)
    if not os.environ.get("LANGSMITH_API_KEY"):
        print("LangSmith setup (optional but recommended for debugging):")
        langsmith_key = getpass.getpass("Enter your LangSmith API Key (or press Enter to skip): ")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"] = langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
            print("✅ LangSmith tracing enabled!")
        else:
            print("⚠️ Skipping LangSmith setup")
    
    print("🚀 Environment setup complete!")

# Run setup
setup_environment()


🚀 Environment setup complete!


## 🧠 Part 1: Understanding LangGraph Fundamentals {#part1}

Before we build anything, let's understand the core concepts that make LangGraph powerful.

### 🔑 Core Concepts

#### 1. **State** 
- The "memory" of your application
- Shared data structure that persists across all steps
- Can contain messages, variables, flags, or any data you need

#### 2. **Nodes**
- Individual functions or operations in your workflow
- Each node receives the current state and returns updates
- Think of them as "workers" that do specific tasks

#### 3. **Edges** 
- Connections between nodes that define the flow
- Can be simple (A → B) or conditional (A → B or C based on logic)

#### 4. **Graph**
- The complete workflow combining nodes and edges
- Defines how your AI agent behaves and makes decisions

### 🎯 Simple Mental Model

Think of LangGraph like a **flowchart for AI agents**:

```
[User Input] → [AI Thinks] → [Uses Tool?] 
                    ↓              ↓
               [Respond]      [Call Tool] → [AI Thinks] → [Respond]
```

But unlike a simple flowchart, LangGraph can:
- Remember everything that happened before
- Loop back to previous steps
- Involve humans in the decision-making
- Handle complex, branching logic

Let's see this in action! 👇


## 🤖 Part 2: Building Your First Simple Agent {#part2}

Let's start with the simplest possible LangGraph application - a basic chatbot that can have a conversation.

### Step 1: Define the State

The state is like the "memory" of our agent. For a chatbot, we need to remember the conversation history.


In [5]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# Define our State - this is the "memory" of our agent
class State(TypedDict):
    # messages will store our conversation history
    # add_messages is a special function that appends new messages instead of replacing them
    messages: Annotated[list, add_messages]

print("✅ State defined!")
print("Our state has one field: 'messages' that will store the conversation history")
print("The add_messages function ensures new messages are appended, not replaced")


✅ State defined!
Our state has one field: 'messages' that will store the conversation history
The add_messages function ensures new messages are appended, not replaced


### Step 2: Create the Language Model

Now we need an AI model to power our chatbot. We'll use OpenAI's GPT model.


In [6]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
    temperature=0.7,        # Controls creativity (0 = deterministic, 1 = very creative)
)

print("✅ Language model initialized!")
print(f"Model: {llm.model_name}")

# Let's test it quickly
test_response = llm.invoke("Say hello in a friendly way!")
print(f"Test response: {test_response.content}")


✅ Language model initialized!
Model: gpt-3.5-turbo
Test response: Hello there! How are you doing today?


### Step 3: Create the Chatbot Node

A **node** is a function that:
1. Takes the current state as input
2. Does some work (like calling the LLM)
3. Returns updates to the state

Let's create our chatbot node:


In [7]:
def chatbot_node(state: State) -> Dict[str, Any]:
    """
    The main chatbot node that processes messages and generates responses.
    
    Args:
        state: Current state containing conversation history
        
    Returns:
        Dict with new messages to add to state
    """
    # Get the conversation history from state
    messages = state["messages"]
    
    # Call the LLM with the conversation history
    response = llm.invoke(messages)
    
    # Return the new message to be added to state
    return {"messages": [response]}

print("✅ Chatbot node created!")
print("This node will:")
print("1. Take the conversation history from state")
print("2. Send it to the LLM")
print("3. Return the LLM's response to be added to state")


✅ Chatbot node created!
This node will:
1. Take the conversation history from state
2. Send it to the LLM
3. Return the LLM's response to be added to state


### Step 4: Build the Graph

Now we'll create the graph by:
1. Creating a StateGraph
2. Adding our chatbot node
3. Defining the flow (edges)
4. Compiling it into a runnable application


In [8]:
# Step 1: Create the graph builder
graph_builder = StateGraph(State)

# Step 2: Add our chatbot node
graph_builder.add_node("chatbot", chatbot_node)

# Step 3: Define the flow
# START -> chatbot -> END
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Step 4: Compile the graph
simple_chatbot = graph_builder.compile()

print("✅ Simple chatbot graph created!")
print("Flow: START → chatbot → END")
print("Ready to chat! 🎉")


✅ Simple chatbot graph created!
Flow: START → chatbot → END
Ready to chat! 🎉


### Step 5: Test Your First LangGraph Agent!

Let's test our simple chatbot. We'll send it a message and see how it responds.


In [12]:
# Test our simple chatbot
def test_simple_chatbot():
    print("🤖 Testing Simple Chatbot")
    print("=" * 40)
    
    # Create initial state with a user message
    initial_state = {
        "messages": [HumanMessage(content="Hello! My name is Niyantarana Tagore. What's your name?")]
    }
    
    # Run the chatbot
    result = simple_chatbot.invoke(initial_state)
    
    # Print the conversation
    for i, message in enumerate(result["messages"]):
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n" + "=" * 40)
    return result

# Run the test
first_result = test_simple_chatbot()


🤖 Testing Simple Chatbot
👤 Human: Hello! My name is Niyantarana Tagore. What's your name?
🤖 AI: Hello Niyantarana Tagore! I'm an AI assistant and I don't have a personal name as I'm here to assist you. How can I help you today?



In [16]:
def test_simple_chatbot(user_input):
    print("🤖 Testing Simple Chatbot")
    print("=" * 40)
    
    initial_state = {
        "messages": [HumanMessage(content=user_input)]
    }
    
    result = simple_chatbot.invoke(initial_state)
    
    for i, message in enumerate(result["messages"]):
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n" + "=" * 40)
    return result


In [18]:
test_simple_chatbot("What is the GATE cutoff for ECE?")

🤖 Testing Simple Chatbot
👤 Human: What is the GATE cutoff for ECE?
🤖 AI: The GATE cutoff for ECE (Electronics and Communication Engineering) can vary each year depending on various factors such as the difficulty of the exam, number of applicants, and the highest score obtained. In general, the cutoff for ECE is usually in the range of 25-35 for general category candidates. However, it is important to note that the cutoff can vary for different institutes and categories of candidates (SC/ST/OBC). It is recommended to check the official GATE website or contact the respective institutes for the most accurate and up-to-date information on cutoff scores.



{'messages': [HumanMessage(content='What is the GATE cutoff for ECE?', additional_kwargs={}, response_metadata={}, id='78f50480-00cc-456a-9d0b-35a816b2b681'),
  AIMessage(content='The GATE cutoff for ECE (Electronics and Communication Engineering) can vary each year depending on various factors such as the difficulty of the exam, number of applicants, and the highest score obtained. In general, the cutoff for ECE is usually in the range of 25-35 for general category candidates. However, it is important to note that the cutoff can vary for different institutes and categories of candidates (SC/ST/OBC). It is recommended to check the official GATE website or contact the respective institutes for the most accurate and up-to-date information on cutoff scores.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 17, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_

In [19]:
test_simple_chatbot("What are the Subjects in GATE ECE?")

🤖 Testing Simple Chatbot
👤 Human: What are the Subjects in GATE ECE?
🤖 AI: The subjects covered in the GATE ECE (Electronics and Communication Engineering) exam are as follows:
1. Engineering Mathematics
2. Networks, Signals, and Systems
3. Electronic Devices
4. Analog Circuits
5. Digital Circuits
6. Control Systems
7. Communications
8. Electromagnetics

These subjects cover a wide range of topics related to electronics and communication engineering and are essential for a thorough understanding of the field.



{'messages': [HumanMessage(content='What are the Subjects in GATE ECE?', additional_kwargs={}, response_metadata={}, id='c5033d8b-6d90-4f93-a8a1-d67ee4334663'),
  AIMessage(content='The subjects covered in the GATE ECE (Electronics and Communication Engineering) exam are as follows:\n1. Engineering Mathematics\n2. Networks, Signals, and Systems\n3. Electronic Devices\n4. Analog Circuits\n5. Digital Circuits\n6. Control Systems\n7. Communications\n8. Electromagnetics\n\nThese subjects cover a wide range of topics related to electronics and communication engineering and are essential for a thorough understanding of the field.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 17, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.

### 🎉 Congratulations!

You just built your first LangGraph agent! But there's a problem...

**The Issue**: Our chatbot doesn't remember previous conversations. Each time we call it, it starts fresh.

Let's test this:


In [20]:
# Test memory issue - the chatbot won't remember the previous conversation
print("🧪 Testing Memory Issue")
print("=" * 40)

# Second message - asking about the name mentioned earlier
second_state = {
    "messages": [HumanMessage(content="What was my name again?")]
}

result2 = simple_chatbot.invoke(second_state)

for message in result2["messages"]:
    if isinstance(message, HumanMessage):
        print(f"👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"🤖 AI: {message.content}")

print("\n❌ As you can see, the chatbot doesn't remember your name!")
print("This is because each call starts with a fresh state.")
print("In the next section, we'll fix this with **memory and checkpointing**! 🧠")


🧪 Testing Memory Issue
👤 Human: What was my name again?
🤖 AI: I'm sorry, I am not able to remember your name as I am a digital assistant and do not have the capability to retain personal information.

❌ As you can see, the chatbot doesn't remember your name!
This is because each call starts with a fresh state.
In the next section, we'll fix this with **memory and checkpointing**! 🧠


## 🧠 Part 3: Adding Memory with Checkpointing {#part3}

The power of LangGraph really shines when we add **persistent memory**. This allows our agent to:

- Remember conversations across multiple interactions
- Resume from where it left off
- Handle long-running workflows
- Support human-in-the-loop scenarios

### What is Checkpointing?

**Checkpointing** automatically saves the state after each step. When you invoke the graph again with the same `thread_id`, it loads the saved state and continues from there.

Think of it like a video game save system! 🎮

### Step 1: Set Up a Checkpointer


In [21]:
!pip install langgraph-checkpoint-sqlite

In [22]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()


print("✅ Checkpointer created!")
print("This will automatically save and restore conversation state")
print("In production, you'd use a real database file instead of ':memory:'")


✅ Checkpointer created!
This will automatically save and restore conversation state
In production, you'd use a real database file instead of ':memory:'


### Step 2: Create a Chatbot with Memory

Now let's rebuild our chatbot with memory capabilities:


In [25]:
graph_builder_with_memory = StateGraph(State)

graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=memory)

In [26]:
import uuid
unique_id = uuid.uuid4()
str(unique_id)
print(f"Unique ID for this conversation: {unique_id}")

Unique ID for this conversation: b8b16daf-62a6-4f62-a13a-b977e234c437


In [27]:
unique_id = "b8b16daf-62a6-4f62-a13a-b977e234c437"
def test_simple(user_input):
    
    config = {"configurable": {"thread_id":str(unique_id)}}
    result = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config
    )
    return result

### Step 3: Test the Memory

The key to using memory is the **config** parameter with a `thread_id`. All conversations with the same `thread_id` will share memory.


In [29]:
def test_memory_chatbot():
    print("🧠 Testing Chatbot with Memory")
    print("=" * 50)
    
    # Configuration with thread_id - this is crucial for memory!
    config = {"configurable": {"thread_id": "conversation_1"}}
    
    # First message
    print("📝 First interaction:")
    result1 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.")]},
        config  # Pass config as second parameter!
    )
    
    for message in result1["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n📝 Second interaction (same thread_id):")
    result2 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="What's my name and what I do?")]},
        config  # Same config = same memory!
    )
    
    # Only show the new messages
    new_messages = result2["messages"][len(result1["messages"]):]
    for message in new_messages:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n✅ Success! The chatbot remembered both your name and Specialization!")
    return result2

# Test the memory
memory_result = test_memory_chatbot()


🧠 Testing Chatbot with Memory
📝 First interaction:
👤 Human: Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.
🤖 AI: Hello Niyantarana Tagore! It's nice to meet you. Can you tell me more about your work as an AI Agent Developer?
👤 Human: What's my name and what I do?
🤖 AI: Your name is Niyantarana Tagore and you are an AI Agent Developer. You specialize in developing artificial intelligence agents to perform tasks and interact with users in various applications. Your work involves designing, programming, and testing AI algorithms to improve the performance and capabilities of these agents.
👤 Human: Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.
🤖 AI: Hello Niyantarana Tagore! It's nice to meet you again. How can I assist you today?

📝 Second interaction (same thread_id):
👤 Human: What's my name and what I do?
🤖 AI: Your name is Niyantarana Tagore and you are an AI Agent Developer. You specialize in creating artificial intelligence agents to perform tas

### 🔍 Understanding Thread IDs

Different `thread_id`s create separate conversations. Let's test this:


In [30]:
# Test with a different thread_id
print("🔄 Testing Different Thread ID")
print("=" * 40)

# Different thread_id = fresh conversation
different_config = {"configurable": {"thread_id": "conversation_2"}}

result_different = chatbot_with_memory.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id!
)

for message in result_different["messages"]:
    if isinstance(message, HumanMessage):
        print(f"👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"🤖 AI: {message.content}")

print("\n🎯 Key Insight:")
print("- Same thread_id = shared memory")
print("- Different thread_id = separate conversations")
print("- This allows multiple users or conversation contexts!")


🔄 Testing Different Thread ID
👤 Human: What's my name?
🤖 AI: I'm sorry, I do not have access to that information.

🎯 Key Insight:
- Same thread_id = shared memory
- Different thread_id = separate conversations
- This allows multiple users or conversation contexts!


### 📊 Inspecting the State

You can inspect the current state of any conversation using `get_state()`:


In [31]:
# Inspect the state of our first conversation
config1 = {"configurable": {"thread_id": "conversation_1"}}
state_snapshot = chatbot_with_memory.get_state(config1)

print("📊 State Inspection")
print("=" * 30)
print(f"Number of messages: {len(state_snapshot.values['messages'])}")
print(f"Next node to execute: {state_snapshot.next}")
print(f"Thread ID: {state_snapshot.config['configurable']['thread_id']}")

print("\n💬 Full conversation history:")
for i, message in enumerate(state_snapshot.values["messages"], 1):
    if isinstance(message, HumanMessage):
        print(f"{i}. 👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"{i}. 🤖 AI: {message.content[:100]}...")  # Truncate for readability


📊 State Inspection
Number of messages: 8
Next node to execute: ()
Thread ID: conversation_1

💬 Full conversation history:
1. 👤 Human: Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.
2. 🤖 AI: Hello Niyantarana Tagore! It's nice to meet you. Can you tell me more about your work as an AI Agent...
3. 👤 Human: What's my name and what I do?
4. 🤖 AI: Your name is Niyantarana Tagore and you are an AI Agent Developer. You specialize in developing arti...
5. 👤 Human: Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.
6. 🤖 AI: Hello Niyantarana Tagore! It's nice to meet you again. How can I assist you today?...
7. 👤 Human: What's my name and what I do?
8. 🤖 AI: Your name is Niyantarana Tagore and you are an AI Agent Developer. You specialize in creating artifi...
